[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/bin/bin_all_features.ipynb)

# Bin - All Features

Complete demo of the Bin widget: synthetic crystal data, interactive binning,
BF/ADF mask tuning, export (PNG/ZIP/GIF), state save/load, and handoff to Show4DSTEM.

In [4]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import numpy as np
import torch
from pathlib import Path
from quantem.widget import Bin

In [ ]:
# --- Synthetic 4D-STEM: SrTiO3 [001] zone axis with asymmetric scan (PyTorch GPU) ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

scan_rows, scan_cols = 64, 64
det_rows, det_cols = 256, 256
pixel_size = (2.39, 2.39)   # Angstrom/px (row, col)
k_pixel_size = (0.46, 0.46)  # mrad/px

# Detector grid
dr = torch.arange(det_rows, device=device, dtype=torch.float32) - det_rows / 2
dc = torch.arange(det_cols, device=device, dtype=torch.float32) - det_cols / 2
KY, KX = torch.meshgrid(dr, dc, indexing='ij')
kr2 = KY**2 + KX**2

# BF disk + amorphous ring
central_beam = torch.exp(-kr2 / (2 * 32.0**2))
amorphous_ring = 0.05 * torch.exp(-(torch.sqrt(kr2) - 90)**2 / (2 * 15**2))

# Bragg spots — perovskite [001] zone axis
lattice_a = 54.0
bragg_spots = torch.zeros(det_rows, det_cols, device=device)
for h in range(-3, 4):
    for k in range(-3, 4):
        if h == 0 and k == 0:
            continue
        dist2 = (KY - h * lattice_a)**2 + (KX - k * lattice_a)**2
        if (h + k) % 2 == 0:
            intensity = 0.4 * torch.exp(torch.tensor(-0.08 * (h**2 + k**2), device=device))
        else:
            intensity = 0.15 * torch.exp(torch.tensor(-0.08 * (h**2 + k**2), device=device))
        bragg_spots += intensity * torch.exp(-dist2 / (2 * 5.0**2))

base_dp = central_beam + amorphous_ring + bragg_spots

# Scan grid — thickness wedge + atom column contrast
si = torch.linspace(0, 1, scan_rows, device=device)
sj = torch.linspace(0, 1, scan_cols, device=device)
SY, SX = torch.meshgrid(si, sj, indexing='ij')
thickness = 1.0 + 0.3 * SY
atom_contrast = 1.0 + 0.2 * torch.sin(2 * torch.pi * SY * 6) * torch.cos(2 * torch.pi * SX * 5)

# Build 4D with broadcasting
modulation = (thickness * atom_contrast).unsqueeze(-1).unsqueeze(-1)
data4d = base_dp.unsqueeze(0).unsqueeze(0) * modulation

# Poisson noise (NumPy — torch.poisson unreliable on MPS)
data_np = data4d.clamp(min=0).cpu().numpy() * 150
data = np.random.default_rng(7).poisson(data_np).astype(np.float32)

size_gb = data.nbytes / 1024**3
print(f'Synthetic 4D-STEM: {data.shape} ({size_gb:.2f} GB)')

In [ ]:
# Create widget with calibration and 8x det binning preset (GPU by default)
w = Bin(
    data,
    pixel_size=pixel_size,
    k_pixel_size=k_pixel_size,
    bin_mode='mean',
    edge_mode='crop',
)
w.scan_bin_row = 2
w.scan_bin_col = 2
w.det_bin_row = 8
w.det_bin_col = 8
w

## Auto-contrast and FFT

Toggle auto-contrast (keyboard `A`) for percentile-clipped display.
Toggle FFT (keyboard `F`) to check Bragg spots survive binning.

In [8]:
# Programmatic auto-contrast + FFT toggle
w.auto_contrast = True
w.show_fft = True
w

Bin(shape=(48, 40, 128, 96), bin=(2, 2, 2, 2), binned_shape=(24, 20, 64, 48), mode=mean, edge=crop, backend=torch, device=mps)

In [9]:
# Export examples
out = Path('bin_exports')
out.mkdir(exist_ok=True)
w.save_image(out / 'bin_grid.png', view='grid')
w.save_zip(out / 'bin_bundle.zip', include_arrays=True)
w.save_gif(out / 'bin_bf_compare.gif', channel='bf')

PosixPath('bin_exports/bin_bf_compare.gif')

In [10]:
# Save preset for batch processing
preset_path = out / 'bin_preset.json'
w.save(preset_path)
w.summary()

Bin
════════════════════════════════
Backend:  torch (mps)
Shape:    (48, 40, 128, 96) -> (24, 20, 64, 48)
Factors:  scan=(2, 2), det=(2, 2), mode=mean, edge=crop
Real cal: (2.39, 2.39) Å/px -> (4.78, 4.78)
K cal:    (0.46, 0.46) mrad/px -> (0.92, 0.92)
Status:   OK - Preview updated on torch/mps: (48×40×128×96) -> (24×20×64×48)


In [11]:
# Reload from saved state
w2 = Bin(data, pixel_size=pixel_size, k_pixel_size=k_pixel_size, state=str(preset_path))
w2.summary()

Bin
════════════════════════════════
Backend:  torch (mps)
Shape:    (48, 40, 128, 96) -> (24, 20, 64, 48)
Factors:  scan=(2, 2), det=(2, 2), mode=mean, edge=crop
Real cal: (2.39, 2.39) Å/px -> (4.78, 4.78)
K cal:    (0.46, 0.46) mrad/px -> (0.92, 0.92)
Status:   OK - Preview updated on torch/mps: (48×40×128×96) -> (24×20×64×48)
